# scikit-learnの様々な分類アルゴリズムを試す
参考URL：[機械学習 iris データセットを用いて scikit-learn の様々な分類アルゴリズムを試してみた](https://qiita.com/ao_log/items/fe9bd42fd249c2a7ee7a)

# irisデータセットのロード

## データセットの説明
データセットの説明は DESCR (description) から見ることができます。

## データセットの形状
説明変数(特徴量)は 4 個で、150 個データがあります。形状は shape で確認できます。

## 花の種類
目的変数は target_names に花の種類が格納されています。 3種類あります。

# データを眺める

## データフレームに変換する
通常データ分析をする時はpandas.DataFrame(データフレーム)に変換します。<br>
pandas(データ解析ライブラリ）により、データが扱いやすくなります。<br>

target の行が 0, 1, 2 だと分かりにくいので種類名に置き換えます。<br>
今回は勉強会用にわかりやすさを重視して置き換えておきます。

ざっくり眺めるには describe です。平均値、最小、最大値が分かります。

# ペアプロット
ペアプロットします。各特徴量のペアごとに散布図を表示させることができます。<br>

## 分類アルゴリズム

学習データの定義

In [ ]:
# import some data to play with
X =    # sepal length, petal length (分類が簡単な組み合わせ。100%になってしまうこともある)
# X =   # sepal length, sepal width (分類が難しい組み合わせ。77%ほどしか精度が出ない)
y = iris.target

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from IPython.display import display

def decision_boundary(clf, X, y, ax, title, report_type="classification"):

    # graph common settings
    h = .02  # step size in the mesh
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    
    # データを訓練データとテストデータに分ける
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)
    
    # 学習する
    clf.fit(X_train, y_train)

    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].    
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    ax.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)

    # Plot also the training points
    ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, edgecolors='k', cmap=plt.cm.Paired)

    # label
    ax.set_title(title)
    ax.set_xlabel('sepal length')
    ax.set_ylabel('petal length')
    
    # 学習データによる精度表示
    if report_type is "classification":
#         print('train classification_report')
        y_true = y_test
        y_pred = clf.predict(X_test)
        report_dict = classification_report(y_true, y_pred,
                                   target_names=["setosa","versicolor","virginica"],
                                   output_dict=True)
        df_report = pd.DataFrame(report_dict)
        display(df_report)
    elif report_type is "regression":
        print('Test score: {:.3f}'.format(clf.score(X_test, y_test)))    

# 教師あり学習
## k-近傍法　(k-NN)
モデルには訓練データセットを格納するだけです。予測時は、予測したいデータポイントの近くの k 個の近傍点を確認します。その中で一番多数派のクラスを予測結果として採用します。

KNeighborsClassifierを使用します。n_neighbors で予測に使用する近傍点の数を設定します。n_neighbors = 1 の場合は決定境界が鋭角になる部分もあります。数が多くなるに従いなだらかになっていき、10 になると境界が単純になりすぎて予測性能が落ちる場合があります。

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

fig, axes = plt.subplots(1, 4, figsize=(12, 3))

for ax, n_neighbors in zip(axes, [1, 3, 6, 10]):
    title = "%s neighbor(s)"% (n_neighbors)
    clf = KNeighborsClassifier(n_neighbors=n_neighbors)
    decision_boundary(clf, X, y, ax, title)

## ロジスティック回帰
名前に回帰とあリますが、分類アルゴリズムです。

LogisticRegressionを使用します。決定境界は直線になります。C は正則化の度合いを調整するパラメータです。正則化は学習時にペナルティを与えることで過学習を抑える効果があります。C を大きくすると正則化が弱くなり過学習気味になりますが、小さすぎるとデータの特徴を大雑把にしか獲得できません。

ロジスティック回帰などの線形モデルは高次元(特徴量の数が多い)のデータに対して有効です。理由は高速だから、他の手法では学習できないからです。

In [ ]:
from sklearn.linear_model import LogisticRegression

fig, axes = plt.subplots(1, 3, figsize=(10, 3))

for ax, C in zip(axes, [0.01, 1, 100]):
    title = "C=%s"% (C)
    clf = LogisticRegression(C=C, solver='lbfgs', multi_class='auto')
    decision_boundary(clf, X, y, ax, title)

C=L2正則化のコスト関数の重み係数<br>
C=100にすると精度が100%に達することがあるが、おそらくオーバーフィッティングしているのでよろしくない。

## 線形サポートベクタマシン
LinearSVCを使用します。線形なので、決定境界も直線になります。<br>
C はロジスティック回帰同様、正則化の度合いを調整するパラメータです。<br>
そのまま生データを使用すると精度が出ないため、StandardScaler()で標準化を行っています<br>
標準化を行うことでデータの平均は０に、分散は１になります。

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
X_std = std.fit_transform(X)

fig, axes = plt.subplots(1, 3, figsize=(10, 3))

for ax, C in zip(axes, [0.01, 1, 100]):
    title = "C=%s"% (C)
    clf = LinearSVC(C=C)
    decision_boundary(clf, X_std, y, ax, title)

## カーネル法を用いたサポートベクタマシン
SVM とも呼ばれます。線形カーネルベクタマシンと比べて、非線形な分離が可能です。<br>
<br>
SVCを使用します。C は正則化のパラメータです、gamma は訓練データの影響が及ぶ範囲で、小さいと遠くまで、大きいと近くになります。よって、gamma が大きすぎると過学習になり、小さすぎると大雑把にしかデータの特徴を獲得できません。<br>
<br>
パラメータの説明は RBF SVM parametersのページにも書かれています。<br>

In [ ]:
from sklearn.svm import SVC

fig, axes = plt.subplots(3, 3, figsize=(10, 10))

for ax_row, C in zip(axes, [0.01, 1, 100]):
    for ax, gamma in zip(ax_row, [0.1, 1, 10]):
        title = "C=%s, gamma=%s"% (C, gamma)
        clf = SVC(C=C, gamma=gamma)
        decision_boundary(clf, X, y, ax, title)

## 決定木
質問を通してデータを分類する手法です。あやめの場合は、花弁の長さが何センチ以上 or 未満、ガクの長さが何センチ以上 or 未満・・・と質問を繰り返すことで品種を特定していきます。

DecisionTreeRegressor を使用します。max_depth がツリーの深さで、質問数になります。多ければいいわけでもなく、訓練データに過剰適合するので過学習となります。

In [ ]:
from sklearn.tree import DecisionTreeRegressor

fig, axes = plt.subplots(1, 4, figsize=(15, 3))

for ax, max_depth in zip(axes, [1, 3, 5, 8]):
    title = "max_depth=%s"% (max_depth)
    clf = DecisionTreeRegressor(max_depth=max_depth)
    decision_boundary(clf, X, y, ax, title, report_type="regression")

## ランダムフォレスト
ランダムフォレストは異なる決定木をたくさん作ります。<br>
全ての決定木で予測した結果からもっとも確率が高くなるラベルを正解とするものです。<br>
個々の木だと過剰適合しているかもしれないですが、多くの結果を集約することで過学習を抑制する効果があります。RandomForestClassifier を使います。<br>

In [ ]:
from sklearn.ensemble import RandomForestClassifier

fig, ax = plt.subplots(1, 1, figsize=(5, 3))
clf = RandomForestClassifier(n_estimators=10)
decision_boundary(clf, X, y, ax, "RandomForestClassifier")

## 勾配ブースティング回帰木
勾配ブースティング回帰木(GBDT = Gradient Boosting Decision Tree)もたくさんの決定木を作るのですが、一つ前の決定木の予測値と正解のズレを修正するように次の木を作っていくのだそうです。GradientBoostingClassifier を使います。<br>
こちらの方がランダムフォレストよりモデル構築に時間がかかったり、パラメータ設定のチューニングが大変らしいのですが、その分予測性能がよくなるのだそうです。

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

fig, ax = plt.subplots(1, 1, figsize=(5, 3))
clf = GradientBoostingClassifier()
decision_boundary(clf, X, y, ax, "GradientBoostingClassifier")

## ニューラルネットワーク
scikit-learn にはニューラルネットワークのライブラリも実装されています。<br>
なお、scikit-learn は GPU に対応していない こともあり、ディープラーニングをやりたい場合は他のライブラリを選定した方がいいと思います。<br>

前置きはさておき、MLPClassifier を使います。<br>
隠れ層 15 個で計算した結果を図示しますが、同じパラメータでも決定境界が異なっています。<br>
そうなる理由は、それぞれ異なる初期状態から学習を開始しているためです。

In [ ]:
from sklearn.neural_network import MLPClassifier

fig, axes = plt.subplots(1, 4, figsize=(12, 3))

for ax, n in zip(axes, [15, 15, 15, 15]):
    title = ""
    clf = MLPClassifier(hidden_layer_sizes=[n, n])
    decision_boundary(clf, X, y, ax, title)

# 教師なし学習
## k-means
教師データなしで分類する手法です。KMeans を使います。n_clustersで何個に分類するかを指定します。

In [ ]:
from sklearn.cluster import KMeans

fig, axes = plt.subplots(1, 4, figsize=(12, 3))

for ax, n_clusters in zip(axes, [2, 3, 4, 5]):
    title = "n_clusters=%s"% (n_clusters)
    clf = KMeans(n_clusters=n_clusters)
    decision_boundary(clf, X, y, ax, title, report_type=False)